<a href="https://colab.research.google.com/github/sagar9926/MTech_Atificial_Intelligence/blob/main/ML1/ML_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1 

## Dataset : Iris Dataset

Use sklearn library for loading iris dataset.

__Aim__: Classification using Naive Bayes classifier

```
● Apply Naive bayes classifier assuming all features are independent.
Do not use any predefined library for classification
Report overall accuracy, class wise accuracy, confusion matrix and ROC curve.
```

In [232]:
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from functools import reduce

### Loading Dataset

In [233]:
iris = datasets.load_iris()
df_iris = pd.DataFrame(iris.data,columns = iris.feature_names)
df_iris['target'] = iris.target

In [234]:
X, y = df_iris.iloc[:, :-1], df_iris.iloc[:, -1]

# # split on train and test 0.7/0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=1, stratify=y)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(75, 4) (75,)
(75, 4) (75,)


In [235]:
df_iris.target.unique()

array([0, 1, 2])

### Creating a Naive Bayes class

In [296]:
class NaiveBayesClassifier:

  """
  Bayes Theorem : 

  P(y|X) = (P(X|y)*P(y))/P(X)

  where :
  X : Input Features
  y : Traget Variable
  P(y|X) = Posterior Probability
  P(X|y) = Liklihood
  P(y) = Prior
  P(X) = Evidence

  """

  def __init__(self,prior = None):

    self.liklihood = {}
    self.posteriors = []
    self.prediction = []
    self.mean = None
    self.variance = None 
    if prior :
      self.prior = prior
    else :
      self.prior = {} 
 
  def calculate_prior(self ,target ,classes):
    """
    This function calculates the prior probabilities
    """
    prior = {}
    for index in classes:
      prior[index] = (target == index).sum()/len(target) 
    return prior


  def feature_statistics(self , features , target):
    """
    This function calculates mean and variance for continuous features
    """
    var = lambda x : np.var(x)  
    mean = features.groupby(target).mean().reset_index()
    #mean = features.groupby(y_train).agg({'sepal length (cm)' : 'mean',	'sepal width (cm)': 'mean',	'petal length (cm)': 'mean',	'petal width (cm)': 'mean'}).reset_index()
    #variance = features.groupby(y_train).agg({'sepal length (cm)' : var,	'sepal width (cm)': var,	'petal length (cm)': var,	'petal width (cm)': var}).reset_index()
    variance = features.groupby(target).var(ddof = 0).reset_index()
    return mean , variance


  def gaussian_probability(self,test_feature,classes, mean, variance):
    liklihood_prob = {}
    for index in classes:
      mean_vector = np.array(mean[mean['target'] == index])[0][1:]
      variance_vector = np.array(variance[variance['target'] == index])[0][1:]
      mean_diff_square = (np.array(test_feature) - mean_vector)**2
      liklihood_prob[index] = (1/np.sqrt(2*np.pi*variance_vector)*np.exp(-1*0.5*mean_diff_square/variance_vector))
    return liklihood_prob


  def calculate_posterior(self,test_feature,classes):

    posteriors = {}
    self.liklihood = self.gaussian_probability(test_feature ,classes ,self.mean,self.variance)
    for index in classes :
      posteriors[index] = reduce(lambda x , y : x*y , self.liklihood[index]) * self.prior[index]
    return posteriors


  def fit(self, features, target):
    self.classes = target.unique()
    if self.prior :
      pass
    else:
      self.prior = self.calculate_prior(target , self.classes)

    
    self.mean , self.variance = self.feature_statistics(features , target)
    

  def predict(self,test_features):
    for i in range(len(test_features)):
      self.posteriors.append(self.calculate_posterior(test_features.iloc[i].values,self.classes))
      self.prediction.append(sorted(self.posteriors[i].items(),key = lambda x : x[1],reverse = True)[0][0])
    #return(self.prediction)

  def accuracy(self , target,names):

    overall_accuracy = sum(target == self.prediction) / len(target)
    print(f"Overall accuracy of data : {round(overall_accuracy*100,2)}%")

    data = pd.DataFrame({'Actual Class' : target.values,'Predicted Class':self.prediction})
    for class_ in target.unique():
      temp = data[data['Actual Class'] == class_]
      print(f"{names[class_]} accuracy of data : {round(sum(temp['Actual Class'] == temp['Predicted Class']) / len(temp)*100,2)}%")



### Model Training

In [297]:
model = NaiveBayesClassifier()
model.fit(X_train, y_train)

In [298]:
model.prior

{0: 0.33707865168539325, 1: 0.39325842696629215, 2: 0.2696629213483146}

### Making predictions using trained model

In [299]:
model.predict(X_test) 

In [300]:
model.accuracy(y_test,iris.target_names)

Overall accuracy of data : 97.75%
virginica accuracy of data : 100.0%
versicolor accuracy of data : 97.22%
setosa accuracy of data : 96.55%


# Question 2 :

__Dataset__: Wine dataset (use sklearn library for loading the dataset)

__Aim__: Naive Bayes Classification
```
Shuffle the data with seed value 42 and perform a 70- 30 stratified split of the data into a train and test set.
Also, plot the class-wise distribution of data in the train and test set (one for train set and one for test set).

Compare the distributions. Now, perform classification as follows:

* Train a Gaussian Naive Bayes classifier and report (a) the class priors, (b) mean and variance of
each feature per class.

* Train another Gaussian Naive Bayes classifier by setting prior probability for the classes. Repeat this
experiment by setting priors in the ratios: (a) 40-40-20 and (b) 80-10-10.
```

In [301]:
wine = datasets.load_wine()
df_wine = pd.DataFrame(wine.data,columns = wine.feature_names)
df_wine['target'] = wine.target

In [302]:
X, y = df_wine.iloc[:, :-1], df_wine.iloc[:, -1]

# # split on train and test 0.7/0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=1, stratify=y)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(89, 13) (89,)
(89, 13) (89,)


In [303]:
df_wine.target.unique()

array([0, 1, 2])

### Model Training

In [304]:
model = NaiveBayesClassifier()
model.fit(X_train, y_train)

### Making predictions using trained model

In [305]:
model.predict(X_test) 

In [306]:
model.accuracy(y_test,wine.target_names)

Overall accuracy of data : 97.75%
class_2 accuracy of data : 100.0%
class_1 accuracy of data : 97.22%
class_0 accuracy of data : 96.55%


In [307]:
gnb = GaussianNB()
from sklearn.metrics import accuracy_score

gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)

print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))

Accuracy: 0.98


In [308]:
model.variance

,target,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0,0.194333,0.534229,0.043891,5.249122,106.405556,0.131023,0.141888,0.006529,0.186544,1.517809,0.013766,0.095725,43211.666667
1,1,0.303494,0.946209,0.099845,9.909469,167.848163,0.335842,0.556179,0.012258,0.202225,0.960318,0.045231,0.289007,22835.742041
2,2,0.209265,1.232503,0.031166,5.631510,145.914931,0.093953,0.092947,0.019675,0.121146,4.726697,0.012591,0.081200,14415.581597
